Получим данные

In [ ]:
import pandas as pd
train = pd.read_csv("train.csv")
train

In [ ]:
train.info()

Как видим, пропущенных значений нет

Получим описательную статистику

In [ ]:
train.describe()

Проверяем каждый принзак на наличие аномалий

In [ ]:
train.value_counts("ClientPeriod")


In [ ]:
train.value_counts("MonthlySpending")

In [ ]:
train.hist("MonthlySpending", bins=50)

In [ ]:
mean_spending = train["MonthlySpending"].mean()
mean_spending

In [ ]:
median_spending = train["MonthlySpending"].quantile()
median_spending

Численные признаки не имеют аномальных значений

Можем убрать признак TotalSpent, так как он линейно зависит от MonthlySpending и ClientPeriod

In [ ]:
train.drop("TotalSpent", axis=1, inplace=True)
train

Посмотрим распределение числовых переменных

In [ ]:
train.hist(bins=50) 

Покажем распределение для категориальных переменных(частотный анализ)

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt

In [ ]:
train_columns = train.columns
for i, column in enumerate(train_columns):
    if i<2 or i>17: # чтобы убрать числовые признаки
        continue
    print(train[column].value_counts())
    print()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(20,20))
axes = axes.flatten()

for i, column in enumerate(train_columns):
    if i<2 or i>17: # числовые признаки чтоб убрать
        continue
    train[column].value_counts().plot.pie(ax=axes[i])
    axes[i].set_title(f'Распределение: {column}', fontsize=10)
    axes[i].set_ylabel('')

for j in range(i, len(axes)):
    axes[j].axis('off')

for k in range(0, 2):
    axes[k].axis("off")

plt.show()

Теперь, закодируем категориальные признаки(используем метод LabelEncoder)

In [ ]:
from sklearn.preprocessing import LabelEncoder

train_encoded = train.copy()

categorial_cols = train.select_dtypes(include=["object", 'category']).columns.tolist()

label_encoders = {} # чтобы для коммита и тестовой выборки использовать те же энкодеры

for col in categorial_cols:
    le = LabelEncoder()
    train_encoded[col] = le.fit_transform(train[col])
    label_encoders[col] = le

train_encoded.describe()



In [ ]:
import seaborn as sns
sns.pairplot(train_encoded)

разделим обучающую выборку на обучающую и контрольную

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(train_encoded[train_encoded.columns[:18]], train_encoded["Churn"], test_size=0.2, shuffle=False)

In [ ]:
train_x


обучаем модель KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 20
knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_model.fit(train_x, train_y)

Получим предсказания модели на тестовой выборке

In [ ]:
l= knn_model.predict(test_x)
l


Проверим качество модели на основе метрики ROС-AUC

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(knn_model.predict(test_x), test_y)

Наша модель KNN предсказывает результат с точностью 0.73

Обучим модель логистической регрессии(максимальное количество итераций 300, так как за меньшее количество модель не достигает сходимости)

In [ ]:
from sklearn.linear_model import LogisticRegression

logr_model = LogisticRegression(max_iter=300)
logr_model.fit(train_x, train_y)

Получим предсказания модели на тестовой выборке и проверим качество модели на основе ROC_AUC

In [ ]:
roc_auc_score(logr_model.predict(test_x), test_y)

Наша модель Logr предсказывает результат с точностью 0.74

Обучим модель SVM(SVC - так как задача классификации)

In [ ]:
from sklearn.svm import LinearSVC

svc_model = LinearSVC(penalty='l2', C=0.3) # с регуляризацией результат лучше на 0.001
svc_model.fit(train_x, train_y)

Получим предсказания модели на тестовой выборке и проверим качество модели на основе ROC_AUC

In [ ]:
roc_auc_score(svc_model.predict(test_x), test_y)

Наша модель SVC предсказывает результат с точостью 0.74(чуть лучше, чем логистическая регрессия)

Обработаем commit-выборку

In [ ]:
check_commit = pd.read_csv("test.csv") #для контрольных предсказаний
check_commit.drop("TotalSpent", axis=1, inplace=True)

check_commit_encoded = check_commit.copy()

for col in categorial_cols:
    le = label_encoders[col]
    check_commit_encoded[col] = le.transform(check_commit[col])

Предскажем значения knn для commit-выборки

In [ ]:
knn_commit_predict = knn_model.predict(check_commit_encoded)

submission = pd.DataFrame({'Id': check_commit.index,
                           'Churn': knn_commit_predict})

submission.to_csv('submission_knn.csv', index=False)



Предскажем значения logr для commit-выборки

In [ ]:
logr_commit_predict = logr_model.predict(check_commit_encoded)

submission = pd.DataFrame({"Id": check_commit.index,
                          'Churn':logr_commit_predict})
submission.to_csv('submission_logr.csv', index=False)

Предскажем значения SVC для commit-выборки

In [ ]:
svc_commit_predict = svc_model.predict(check_commit_encoded)

submission = pd.DataFrame({"Id": check_commit.index,
                          'Churn':svc_commit_predict})
submission.to_csv('submission_svc.csv', index=False)